# Run in am to see if MAE improves

In [7]:
import pandas as pd
import random
import pandas as pd
import numpy as np
import pickle

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
%matplotlib inline

from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise import Dataset
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy

from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import KFold
from surprise import NormalPredictor

from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV



In [8]:
ratings_api = pd.read_pickle('user_book_ratings.pkl')
ratings_bs = pd.read_pickle('user_book_ratings_scraped.pkl')
ratings = ratings_api + ratings_bs
len(ratings)

5902

5902

5902

5902

5902

In [9]:
import requests
from bs4 import BeautifulSoup
def get_soup(url):
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    #print(soup.prettify())
    return(soup)
def get_count(soup):
    count = soup.find_all('div',
                          {'class': 'inter loading uitext'})[1].text.split('of ')[1].split(' loaded')[0]

def get_maxpage(soup):
    page_links = []
    for link in soup.find_all('a'):
            #links.append(link['href'])
        try:
            if '?page=' in str(link['href']):
                page_links.append(str(link['href']))
        except:
            pass

    max_page = page_links[-2].split('?page=')[1]
    return(max_page)
    #print(max_page, count)

gid = '69976309'
ratings_user_i = {} # user_ratings_dict = {'userid':gid, 'ratings': user_books_rating_list}
ratings_user_i['userid'] = gid
    
try:
    soup1 = get_soup('https://www.goodreads.com/review/list/'+gid)
    max_page = get_maxpage(soup1)  

    user_ratings_all = []

    for page in range(1, int(max_page)):

        url = 'https://www.goodreads.com/review/list/'+gid+'?page='+str(page)
        soup = get_soup(url)

        tables = soup.find_all("table")
            #tables[1]
        rows = tables[1].find_all('tr')

        tds = [row.find_all('td') for row in rows]

        for i, td in enumerate(tds):
            user_ratings = {}
            try:
                user_ratings['rating'] = tds[i][13].find('span', {'class': 'staticStars'}).text
                user_ratings['bookid'] = tds[i][2].find('div', 
                                    {'class': "js-tooltipTrigger tooltipTrigger"})['data-resource-id']
                user_ratings_all.append(user_ratings)
            except:
                pass
        ratings_user_i['ratings'] = user_ratings_all
except:
    pass

ratings.append(ratings_user_i)
    

In [10]:
d = ratings[10]
d

{'userid': '23380342',
 'ratings': [{'bookid': '35153401', 'rating': '0'},
  {'bookid': '34221184', 'rating': '1'},
  {'bookid': '11228096', 'rating': '0'},
  {'bookid': '35798610', 'rating': '0'},
  {'bookid': '34523045', 'rating': '5'},
  {'bookid': '31942609', 'rating': '5'},
  {'bookid': '26020645', 'rating': '5'},
  {'bookid': '26020624', 'rating': '5'},
  {'bookid': '26020612', 'rating': '5'},
  {'bookid': '26020546', 'rating': '5'},
  {'bookid': '11874787', 'rating': '5'},
  {'bookid': '30130556', 'rating': '5'},
  {'bookid': '30130492', 'rating': '5'},
  {'bookid': '30130418', 'rating': '5'},
  {'bookid': '30130155', 'rating': '5'},
  {'bookid': '42943467', 'rating': '4'},
  {'bookid': '30130385', 'rating': '5'},
  {'bookid': '30130309', 'rating': '5'},
  {'bookid': '30130284', 'rating': '5'},
  {'bookid': '30130247', 'rating': '5'},
  {'bookid': '21461559', 'rating': '5'},
  {'bookid': '41418286', 'rating': '1'},
  {'bookid': '41034133', 'rating': '1'},
  {'bookid': '30758289'

{'userid': '23380342',
 'ratings': [{'bookid': '35153401', 'rating': '0'},
  {'bookid': '34221184', 'rating': '1'},
  {'bookid': '11228096', 'rating': '0'},
  {'bookid': '35798610', 'rating': '0'},
  {'bookid': '34523045', 'rating': '5'},
  {'bookid': '31942609', 'rating': '5'},
  {'bookid': '26020645', 'rating': '5'},
  {'bookid': '26020624', 'rating': '5'},
  {'bookid': '26020612', 'rating': '5'},
  {'bookid': '26020546', 'rating': '5'},
  {'bookid': '11874787', 'rating': '5'},
  {'bookid': '30130556', 'rating': '5'},
  {'bookid': '30130492', 'rating': '5'},
  {'bookid': '30130418', 'rating': '5'},
  {'bookid': '30130155', 'rating': '5'},
  {'bookid': '42943467', 'rating': '4'},
  {'bookid': '30130385', 'rating': '5'},
  {'bookid': '30130309', 'rating': '5'},
  {'bookid': '30130284', 'rating': '5'},
  {'bookid': '30130247', 'rating': '5'},
  {'bookid': '21461559', 'rating': '5'},
  {'bookid': '41418286', 'rating': '1'},
  {'bookid': '41034133', 'rating': '1'},
  {'bookid': '30758289'

{'userid': '23380342',
 'ratings': [{'bookid': '35153401', 'rating': '0'},
  {'bookid': '34221184', 'rating': '1'},
  {'bookid': '11228096', 'rating': '0'},
  {'bookid': '35798610', 'rating': '0'},
  {'bookid': '34523045', 'rating': '5'},
  {'bookid': '31942609', 'rating': '5'},
  {'bookid': '26020645', 'rating': '5'},
  {'bookid': '26020624', 'rating': '5'},
  {'bookid': '26020612', 'rating': '5'},
  {'bookid': '26020546', 'rating': '5'},
  {'bookid': '11874787', 'rating': '5'},
  {'bookid': '30130556', 'rating': '5'},
  {'bookid': '30130492', 'rating': '5'},
  {'bookid': '30130418', 'rating': '5'},
  {'bookid': '30130155', 'rating': '5'},
  {'bookid': '42943467', 'rating': '4'},
  {'bookid': '30130385', 'rating': '5'},
  {'bookid': '30130309', 'rating': '5'},
  {'bookid': '30130284', 'rating': '5'},
  {'bookid': '30130247', 'rating': '5'},
  {'bookid': '21461559', 'rating': '5'},
  {'bookid': '41418286', 'rating': '1'},
  {'bookid': '41034133', 'rating': '1'},
  {'bookid': '30758289'

{'userid': '23380342',
 'ratings': [{'bookid': '35153401', 'rating': '0'},
  {'bookid': '34221184', 'rating': '1'},
  {'bookid': '11228096', 'rating': '0'},
  {'bookid': '35798610', 'rating': '0'},
  {'bookid': '34523045', 'rating': '5'},
  {'bookid': '31942609', 'rating': '5'},
  {'bookid': '26020645', 'rating': '5'},
  {'bookid': '26020624', 'rating': '5'},
  {'bookid': '26020612', 'rating': '5'},
  {'bookid': '26020546', 'rating': '5'},
  {'bookid': '11874787', 'rating': '5'},
  {'bookid': '30130556', 'rating': '5'},
  {'bookid': '30130492', 'rating': '5'},
  {'bookid': '30130418', 'rating': '5'},
  {'bookid': '30130155', 'rating': '5'},
  {'bookid': '42943467', 'rating': '4'},
  {'bookid': '30130385', 'rating': '5'},
  {'bookid': '30130309', 'rating': '5'},
  {'bookid': '30130284', 'rating': '5'},
  {'bookid': '30130247', 'rating': '5'},
  {'bookid': '21461559', 'rating': '5'},
  {'bookid': '41418286', 'rating': '1'},
  {'bookid': '41034133', 'rating': '1'},
  {'bookid': '30758289'

{'userid': '23380342',
 'ratings': [{'bookid': '35153401', 'rating': '0'},
  {'bookid': '34221184', 'rating': '1'},
  {'bookid': '11228096', 'rating': '0'},
  {'bookid': '35798610', 'rating': '0'},
  {'bookid': '34523045', 'rating': '5'},
  {'bookid': '31942609', 'rating': '5'},
  {'bookid': '26020645', 'rating': '5'},
  {'bookid': '26020624', 'rating': '5'},
  {'bookid': '26020612', 'rating': '5'},
  {'bookid': '26020546', 'rating': '5'},
  {'bookid': '11874787', 'rating': '5'},
  {'bookid': '30130556', 'rating': '5'},
  {'bookid': '30130492', 'rating': '5'},
  {'bookid': '30130418', 'rating': '5'},
  {'bookid': '30130155', 'rating': '5'},
  {'bookid': '42943467', 'rating': '4'},
  {'bookid': '30130385', 'rating': '5'},
  {'bookid': '30130309', 'rating': '5'},
  {'bookid': '30130284', 'rating': '5'},
  {'bookid': '30130247', 'rating': '5'},
  {'bookid': '21461559', 'rating': '5'},
  {'bookid': '41418286', 'rating': '1'},
  {'bookid': '41034133', 'rating': '1'},
  {'bookid': '30758289'

In [11]:
len(ratings)

5903

5903

5903

5903

5903

In [12]:
df = pd.DataFrame(ratings[0]['ratings']).assign(userid = ratings[0]['userid'])
df

,userid


,userid


,userid


,userid


,userid


In [13]:
df = pd.DataFrame(ratings[0]['ratings']).assign(userid = ratings[0]['userid'])
for i,d in enumerate(ratings[1:]):
    print(i, d['userid'])
    df = pd.concat([df, pd.DataFrame(d['ratings']).assign(userid = d['userid'])])
    if i%50==0:
        pd.to_pickle(df, 'user_product_long_v2.pkl')
pd.to_pickle(df, 'user_product_long_v2.pkl')

0 16399833
1 23674862
2 45800365
3 18587869
4 2900812
5 2944417
6 34028324
7 10158841
8 2971546
9 23380342
10 2654460
11 6197300
12 4206376
13 5796888
14 26566327
15 3233576
16 5430781
17 23019397
18 11605103
19 5455542
20 7340689
21 12489794
22 1202018
23 3884651
24 35669997
25 52807
26 6173589
27 23133809
28 63407434
29 23841736
30 16399833
31 23674862
32 45800365
33 18587869
34 2900812
35 2944417
36 34028324
37 10158841
38 2971546
39 23380342
40 2654460
41 6197300
42 4206376
43 57968880 16399833
1 23674862
2 45800365
3 18587869
4 2900812
5 2944417
6 34028324
7 10158841
8 2971546
9 23380342
10 2654460
11 6197300
12 4206376
13 5796888
14 26566327
15 3233576
16 5430781
17 23019397
18 11605103
19 5455542
20 7340689
21 12489794
22 1202018
23 3884651
24 35669997
25 52807
26 6173589
27 23133809
28 63407434
29 23841736
30 16399833
31 23674862
32 45800365
33 18587869
34 2900812
35 2944417
36 34028324
37 10158841
38 2971546
39 23380342
40 2654460
41 6197300
42 4206376
43 57968880 16399833
1 2

143 23994847
144 45762620
145 4193760
146 20912853
147 44759659
148 10015197
149 24879665
150 2876676
151 35008458
152 3678654
143 23994847
144 45762620
145 4193760
146 20912853
147 44759659
148 10015197
149 24879665
150 2876676
151 35008458
152 3678654
143 23994847
144 45762620
145 4193760
146 20912853
147 44759659
148 10015197
149 24879665
150 2876676
151 35008458
152 3678654
143 23994847
144 45762620
145 4193760
146 20912853
147 44759659
148 10015197
149 24879665
150 2876676
151 35008458
152 3678654
143 23994847
144 45762620
145 4193760
146 20912853
147 44759659
148 10015197
149 24879665
150 2876676
151 35008458
152 3678654
153 2592041
154 19837052
155 1033929
156 6947675
157 28512145
158 3190472
159 5734529
160 11728126
161 6333257
153 2592041
154 19837052
155 1033929
156 6947675
157 28512145
158 3190472
159 5734529
160 11728126
161 6333257
153 2592041
154 19837052
155 1033929
156 6947675
157 28512145
158 3190472
159 5734529
160 11728126
161 6333257
153 2592041
154 19837052
155 103

275 3082442
276 5368345
277 3415630
278 12019541
279 33055871
280 44385965
281 20514385
282 5741885
283 18786838
284 4810289
285 6156034
286 6599279
287 1068587
288 7766808
289 76271043
290 73315287
291 32124722
292 7242712
293 89885640
294 19158868
275 3082442
276 5368345
277 3415630
278 12019541
279 33055871
280 44385965
281 20514385
282 5741885
283 18786838
284 4810289
285 6156034
286 6599279
287 1068587
288 7766808
289 76271043
290 73315287
291 32124722
292 7242712
293 89885640
294 19158868
275 3082442
276 5368345
277 3415630
278 12019541
279 33055871
280 44385965
281 20514385
282 5741885
283 18786838
284 4810289
285 6156034
286 6599279
287 1068587
288 7766808
289 76271043
290 73315287
291 32124722
292 7242712
293 89885640
294 19158868
275 3082442
276 5368345
277 3415630
278 12019541
279 33055871
280 44385965
281 20514385
282 5741885
283 18786838
284 4810289
285 6156034
286 6599279
287 1068587
288 7766808
289 76271043
290 73315287
291 32124722
292 7242712
293 89885640
294 19158868


409 4352179
410 2134374
411 1797862
412 44584399
413 13911840
414 14378504
415 85322981
416 57162430
401 12703769
402 49346237
403 22209995
404 14649015
405 36472307
406 3951008
407 9546503
408 18848295
409 4352179
410 2134374
411 1797862
412 44584399
413 13911840
414 14378504
415 85322981
416 57162430
401 12703769
402 49346237
403 22209995
404 14649015
405 36472307
406 3951008
407 9546503
408 18848295
409 4352179
410 2134374
411 1797862
412 44584399
413 13911840
414 14378504
415 85322981
416 57162430
401 12703769
402 49346237
403 22209995
404 14649015
405 36472307
406 3951008
407 9546503
408 18848295
409 4352179
410 2134374
411 1797862
412 44584399
413 13911840
414 14378504
415 85322981
416 57162430
417 13906862
418 35621973
419 35192578
420 25267028
421 59412023
422 647547
423 32589419
424 65866051
417 13906862
418 35621973
419 35192578
420 25267028
421 59412023
422 647547
423 32589419
424 65866051
417 13906862
418 35621973
419 35192578
420 25267028
421 59412023
422 647547
423 325894

531 36283432
532 43939519
533 1454825
534 38134833
535 32401998
536 14264125
537 2390510
538 826035
539 1425120
540 20591128
541 34799235
542 39790379
543 25642724
531 36283432
532 43939519
533 1454825
534 38134833
535 32401998
536 14264125
537 2390510
538 826035
539 1425120
540 20591128
541 34799235
542 39790379
543 25642724
531 36283432
532 43939519
533 1454825
534 38134833
535 32401998
536 14264125
537 2390510
538 826035
539 1425120
540 20591128
541 34799235
542 39790379
543 25642724
544 4904968
545 1245002
546 3819215
547 798147
548 6243741
549 90488249
550 3364087
549 90488249
550 3364087
544 4904968
545 1245002
546 3819215
547 798147
548 6243741
544 4904968
545 1245002
546 3819215
547 798147
548 6243741
549 90488249
550 3364087
544 4904968
545 1245002
546 3819215
547 798147
548 6243741
549 90488249
550 3364087
549 90488249
550 3364087
551 42720825
552 66532602
553 3330699
554 30847851
555 8427779
556 3096157
557 32225725
558 310539
559 8437232
560 20617120
551 42720825
552 665326

...

5295 5617611
5296 56872266
5295 5617611
5296 56872266
5295 5617611
5296 56872266
5295 5617611
5296 56872266
5297 40398224
5298 8891323
5297 40398224
5298 8891323
5297 40398224
5298 8891323
5297 40398224
5298 8891323
5297 40398224
5298 8891323
5299 31407964
5300 9339410
5299 31407964
5300 9339410
5299 31407964
5300 9339410
5299 31407964
5300 9339410
5299 31407964
5300 9339410
5301 7959224
5302 32390918
5301 7959224
5302 32390918
5301 7959224
5302 32390918
5301 7959224
5302 32390918
5301 7959224
5302 32390918
5303 147328
5304 6044402
5303 147328
5304 6044402
5303 147328
5304 6044402
5303 147328
5304 6044402
5303 147328
5304 6044402
5305 8744786
5306 2463932
5307 6553564
5308 17144484
5305 8744786
5306 2463932
5307 6553564
5308 17144484
5305 8744786
5306 2463932
5307 6553564
5308 17144484
5305 8744786
5306 2463932
5307 6553564
5308 17144484
5305 8744786
5306 2463932
5307 6553564
5308 17144484
5309 25621220
5310 73083018
5309 25621220
5310 73083018
5311 59033840
5312 31233316
5311 59033840

5417 4648470
5418 44195641
5417 4648470
5418 44195641
5419 6205802
5420 3462402
5419 6205802
5420 3462402
5417 4648470
5418 44195641
5419 6205802
5420 3462402
5417 4648470
5418 44195641
5419 6205802
5420 3462402
5417 4648470
5418 44195641
5419 6205802
5420 3462402
5421 50692307
5422 766700
5421 50692307
5422 766700
5421 50692307
5422 766700
5421 50692307
5422 766700
5421 50692307
5422 766700
5423 32134713
5424 4468872
5425 93628050
5426 5725268
5423 32134713
5424 4468872
5425 93628050
5426 5725268
5423 32134713
5424 4468872
5425 93628050
5426 5725268
5423 32134713
5424 4468872
5425 93628050
5426 5725268
5423 32134713
5424 4468872
5425 93628050
5426 5725268
5427 5652460
5428 5217990
5427 5652460
5428 5217990
5427 5652460
5428 5217990
5427 5652460
5428 5217990
5429 5986718
5430 3451947
5427 5652460
5428 5217990
5429 5986718
5430 3451947
5429 5986718
5430 3451947
5429 5986718
5430 3451947
5429 5986718
5430 3451947
5431 358429
5432 5259019
5431 358429
5432 5259019
5433 78250
5434 22224850


5539 19209473
5540 38622506
5539 19209473
5540 38622506
5539 19209473
5540 38622506
5539 19209473
5540 38622506
5539 19209473
5540 38622506
5541 15362074
5542 1322529
5541 15362074
5542 1322529
5541 15362074
5542 1322529
5541 15362074
5542 1322529
5541 15362074
5542 1322529
5543 8011508
5544 37712806
5543 8011508
5544 37712806
5545 77408024
5546 44235074
5545 77408024
5546 44235074
5543 8011508
5544 37712806
5543 8011508
5544 37712806
5543 8011508
5544 37712806
5545 77408024
5546 44235074
5545 77408024
5546 44235074
5545 77408024
5546 44235074
5547 33440468
5548 65196391
5547 33440468
5548 65196391
5549 7708778
5550 26111385
5549 7708778
5550 26111385
5547 33440468
5548 65196391
5549 7708778
5550 26111385
5547 33440468
5548 65196391
5549 7708778
5550 26111385
5547 33440468
5548 65196391
5549 7708778
5550 26111385
5551 6457194
5552 53687523
5551 6457194
5552 53687523
5551 6457194
5552 53687523
5551 6457194
5552 53687523
5551 6457194
5552 53687523
5553 55110601
5554 10121927
5553 5511060

5659 11170160
5660 82515745
5659 11170160
5660 82515745
5661 73323654
5662 16620145
5661 73323654
5662 16620145
5661 73323654
5662 16620145
5661 73323654
5662 16620145
5661 73323654
5662 16620145
5663 18161411
5664 7410667
5663 18161411
5664 7410667
5663 18161411
5664 7410667
5663 18161411
5664 7410667
5663 18161411
5664 7410667
5665 6559910
5666 16006394
5665 6559910
5666 16006394
5665 6559910
5666 16006394
5665 6559910
5666 16006394
5665 6559910
5666 16006394
5667 4372877
5668 37886652
5669 38486638
5670 90121829
5667 4372877
5668 37886652
5669 38486638
5670 90121829
5667 4372877
5668 37886652
5667 4372877
5668 37886652
5667 4372877
5668 37886652
5671 9823340
5672 80437
5671 9823340
5672 80437
5669 38486638
5670 90121829
5671 9823340
5672 80437
5669 38486638
5670 90121829
5671 9823340
5672 80437
5669 38486638
5670 90121829
5671 9823340
5672 80437
5673 16468369
5674 35863123
5673 16468369
5674 35863123
5675 18634487
5676 56529469
5675 18634487
5676 56529469
5673 16468369
5674 35863123

5781 1078236
5782 5673713
5781 1078236
5782 5673713
5781 1078236
5782 5673713
5783 14529007
5784 11936412
5783 14529007
5784 11936412
5785 41317900
5786 12696619
5785 41317900
5786 12696619
5783 14529007
5784 11936412
5785 41317900
5786 12696619
5783 14529007
5784 11936412
5785 41317900
5786 12696619
5783 14529007
5784 11936412
5785 41317900
5786 12696619
5787 16352892
5788 6988492
5787 16352892
5788 6988492
5787 16352892
5788 6988492
5787 16352892
5788 6988492
5787 16352892
5788 6988492
5789 58814929
5790 4291964
5789 58814929
5790 4291964
5789 58814929
5790 4291964
5789 58814929
5790 4291964
5789 58814929
5790 4291964
5791 8930258
5792 25190617
5791 8930258
5792 25190617
5793 1421462
5794 36190171
5793 1421462
5794 36190171
5791 8930258
5792 25190617
5793 1421462
5794 36190171
5791 8930258
5792 25190617
5793 1421462
5794 36190171
5791 8930258
5792 25190617
5793 1421462
5794 36190171
5795 15895358
5796 41005858
5795 15895358
5796 41005858
5795 15895358
5796 41005858
5795 15895358
5796

In [14]:
len(list(df['userid'].unique()))

5737

5737

5737

5737

5737

In [15]:
df = pd.read_pickle('user_product_long_v2.pkl')

In [16]:
df[df['bookid']=='10664113']

,bookid,rating,userid
759,10664113,0,45800365
91,10664113,0,10158841
8,10664113,4,26566327
10533,10664113,4,23133809
759,10664113,0,45800365
91,10664113,0,10158841
8,10664113,4,26566327
2328,10664113,0,4107783
1541,10664113,3,76129
121,10664113,3,9828786


,bookid,rating,userid
759,10664113,0,45800365
91,10664113,0,10158841
8,10664113,4,26566327
10533,10664113,4,23133809
759,10664113,0,45800365
91,10664113,0,10158841
8,10664113,4,26566327
2328,10664113,0,4107783
1541,10664113,3,76129
121,10664113,3,9828786


,bookid,rating,userid
759,10664113,0,45800365
91,10664113,0,10158841
8,10664113,4,26566327
10533,10664113,4,23133809
759,10664113,0,45800365
91,10664113,0,10158841
8,10664113,4,26566327
2328,10664113,0,4107783
1541,10664113,3,76129
121,10664113,3,9828786


,bookid,rating,userid
759,10664113,0,45800365
91,10664113,0,10158841
8,10664113,4,26566327
10533,10664113,4,23133809
759,10664113,0,45800365
91,10664113,0,10158841
8,10664113,4,26566327
2328,10664113,0,4107783
1541,10664113,3,76129
121,10664113,3,9828786


,bookid,rating,userid
759,10664113,0,45800365
91,10664113,0,10158841
8,10664113,4,26566327
10533,10664113,4,23133809
759,10664113,0,45800365
91,10664113,0,10158841
8,10664113,4,26566327
2328,10664113,0,4107783
1541,10664113,3,76129
121,10664113,3,9828786


**Final cleaning**

In [17]:
df.loc[df['rating'] == 'it was amazing', 'rating'] = 5
df.loc[df['rating'] == 'really liked it', 'rating'] = 4
df.loc[df['rating'] == 'liked it', 'rating'] = 3
df.loc[df['rating'] == 'it was ok', 'rating'] = 2
df.loc[df['rating'] == 'did not like it', 'rating'] = 1
df = df.apply(pd.to_numeric, errors='coerce')
df = df[df['rating'].isna() == False].sort_values(by='userid')

In [18]:
book_titles = pd.read_pickle('book_titles_map.pkl')
df['bookid'] = [str(int(X)) for X  in df['bookid']]
df['userid'] = [int(X) for X  in df['userid']]
df['title'] = df['bookid'].map(book_titles)

In [19]:
df['rating'][df['rating'] != 0].value_counts()

4.0    711786
5.0    653375
3.0    448868
2.0    125226
1.0     40075
Name: rating, dtype: int64

4.0    711786
5.0    653375
3.0    448868
2.0    125226
1.0     40075
Name: rating, dtype: int64

4.0    711786
5.0    653375
3.0    448868
2.0    125226
1.0     40075
Name: rating, dtype: int64

4.0    711786
5.0    653375
3.0    448868
2.0    125226
1.0     40075
Name: rating, dtype: int64

4.0    711786
5.0    653375
3.0    448868
2.0    125226
1.0     40075
Name: rating, dtype: int64

In [20]:
books = pd.read_pickle('book_ids.pkl')
book_cols = list(set(books))
df = df[df['bookid'].isin(book_cols)]
df['title'] = df['bookid'].map(book_titles)

In [21]:
droplist = ['2657','4667024','77203','19063','1984','30119','18143977','18007564','32075671',
            '5470','8664353','10210','7445','19543','23772','19486412','8127','14935','332613',
            '18774964', '113946','11486','191139','77727','4406','21853621','7784','14891',
            '43070','99107','1420','168668','233093','11566','11422','99107','10210'
           ]

df = df[df['bookid'].isin(droplist) == False]
df = df[df['title'].isna() == False]
df = df[df['rating'].isna() == False]
df = df[df['rating'] != 0]

In [22]:
books = pd.read_pickle('books_all.pkl')
books_rating_aggs = []
books_rating_aggs.append([{'bookid': b['id'],
                           'average_rating': b['average_rating'],
                           'ratings_count': b['ratings_count'],
                           'title': b['title']
                          } for b in books])
books_ratings_aggs = pd.DataFrame.from_dict(books_rating_aggs[0])
books_ratings_aggs['average_rating'] = [float(X) for X in books_ratings_aggs['average_rating']]
books_ratings_aggs['ratings_count'] = [float(X) for X in books_ratings_aggs['ratings_count']]
books_ratings_aggs = books_ratings_aggs[books_ratings_aggs['bookid'].isin(droplist) == False]

In [23]:
pd.to_pickle(df, 'user_product_long_v2.pkl')

**Should I drop books with low count or not?**

In [24]:
df['book_count'] = df['bookid'].map(df['bookid'].value_counts())
df['bookid'].value_counts().describe()

count    29974.000000
mean        20.237439
std         53.265212
min          1.000000
25%          2.000000
50%          6.000000
75%         18.000000
max       1506.000000
Name: bookid, dtype: float64

count    29974.000000
mean        20.237439
std         53.265212
min          1.000000
25%          2.000000
50%          6.000000
75%         18.000000
max       1506.000000
Name: bookid, dtype: float64

count    29974.000000
mean        20.237439
std         53.265212
min          1.000000
25%          2.000000
50%          6.000000
75%         18.000000
max       1506.000000
Name: bookid, dtype: float64

count    29974.000000
mean        20.237439
std         53.265212
min          1.000000
25%          2.000000
50%          6.000000
75%         18.000000
max       1506.000000
Name: bookid, dtype: float64

count    29974.000000
mean        20.237439
std         53.265212
min          1.000000
25%          2.000000
50%          6.000000
75%         18.000000
max       1506.000000
Name: bookid, dtype: float64

In [25]:
#df = df[df['book_count'] >= 5]
#pd.to_pickle(df, 'user_product_long_50pct.pkl')

**Surprise dataset**

In [26]:
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userid', 'bookid', 'rating']], reader)

In [27]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), 
                  KNNBasic(), KNNWithMeans(), KNNWithZScore(), KNNBaseline(), 
                  NormalPredictor(), CoClustering()]:
#                  NMF()
# SVDpp(), SlopeOne(), SlopeOne(),
# BaselineOnly()
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE', 'MAE'], cv=3, verbose=False)    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    print(algorithm,'done')

cross_val_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_mae')

pd.to_pickle(cross_val_results, 'rec_model_comparison.pkl') 

cross_val_results


<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7fb270d056a0> done
<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7fb270d056a0> done
<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7fb270d056a0> done
<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7fb270d056a0> done
<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7fb270d056a0> done
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.
Done 

,fit_time,test_mae,test_rmse,test_time
Algorithm,,,,
KNNWithZScore,3.244706,0.659935,0.864562,20.842762
KNNBaseline,4.399086,0.661796,0.859268,22.166353
KNNWithMeans,3.024792,0.662564,0.863984,21.333678
SVD,18.496995,0.665203,0.858953,1.203914
CoClustering,8.615148,0.691474,0.897945,0.903818
KNNBasic,2.892651,0.729263,0.935648,19.061474
NormalPredictor,0.556513,1.042883,1.318342,2.486002


,fit_time,test_mae,test_rmse,test_time
Algorithm,,,,
KNNWithZScore,3.244706,0.659935,0.864562,20.842762
KNNBaseline,4.399086,0.661796,0.859268,22.166353
KNNWithMeans,3.024792,0.662564,0.863984,21.333678
SVD,18.496995,0.665203,0.858953,1.203914
CoClustering,8.615148,0.691474,0.897945,0.903818
KNNBasic,2.892651,0.729263,0.935648,19.061474
NormalPredictor,0.556513,1.042883,1.318342,2.486002


,fit_time,test_mae,test_rmse,test_time
Algorithm,,,,
KNNWithZScore,3.244706,0.659935,0.864562,20.842762
KNNBaseline,4.399086,0.661796,0.859268,22.166353
KNNWithMeans,3.024792,0.662564,0.863984,21.333678
SVD,18.496995,0.665203,0.858953,1.203914
CoClustering,8.615148,0.691474,0.897945,0.903818
KNNBasic,2.892651,0.729263,0.935648,19.061474
NormalPredictor,0.556513,1.042883,1.318342,2.486002


,fit_time,test_mae,test_rmse,test_time
Algorithm,,,,
KNNWithZScore,3.244706,0.659935,0.864562,20.842762
KNNBaseline,4.399086,0.661796,0.859268,22.166353
KNNWithMeans,3.024792,0.662564,0.863984,21.333678
SVD,18.496995,0.665203,0.858953,1.203914
CoClustering,8.615148,0.691474,0.897945,0.903818
KNNBasic,2.892651,0.729263,0.935648,19.061474
NormalPredictor,0.556513,1.042883,1.318342,2.486002


,fit_time,test_mae,test_rmse,test_time
Algorithm,,,,
KNNWithZScore,3.244706,0.659935,0.864562,20.842762
KNNBaseline,4.399086,0.661796,0.859268,22.166353
KNNWithMeans,3.024792,0.662564,0.863984,21.333678
SVD,18.496995,0.665203,0.858953,1.203914
CoClustering,8.615148,0.691474,0.897945,0.903818
KNNBasic,2.892651,0.729263,0.935648,19.061474
NormalPredictor,0.556513,1.042883,1.318342,2.486002


**Gridsearch over top algorithms**

*Comment out whichever wasn't top baseline*

In [33]:
svd_param_grid = {'n_epochs': [20, 30, 50], 'lr_all': [0.001, 0.004, 0.008],
                  'reg_all': [0.05, 0.1, 0.3]}
gs = GridSearchCV(SVD, svd_param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)
# best RMSE, MAE score
print('best mae: ',gs.best_score['mae'])
print('best mae params: ', gs.best_params['mae'])
print('best rmse: ',gs.best_score['rmse'])
print('best rmse params: ', gs.best_params['rmse'])

KeyboardInterrupt: 

KeyboardInterrupt: 

KeyboardInterrupt: 

KeyboardInterrupt: 

KeyboardInterrupt: 

In [32]:
knn_param_grid = {'k': [15, 20, 25, 30, 40, 50, 60]}
gs = GridSearchCV(KNNWithZScore, knn_param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)
print('best mae: ',gs.best_score['mae'])
print('best mae params: ', gs.best_params['mae'])
print('best rmse: ',gs.best_score['rmse'])
print('best rmse params: ', gs.best_params['rmse'])


Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Done co

In [30]:
trainset, testset = train_test_split(data, test_size=0.25, random_state = 88)

svd = SVD(n_epochs=gs.best_params['mae']['n_epochs'], 
           lr_all=gs.best_params['mae']['lr_all'], 
           reg_all=gs.best_params['mae']['reg_all']
          )

#cross_validate(algo, trainset, measures=['RMSE'], cv=3, verbose=False)
algo = svd.fit(trainset)

#test_pred = algo.test(testset)

predictions = algo.test(testset)

print("SVD : Test Set")
print(accuracy.rmse(predictions, verbose=True))
print(accuracy.mae(predictions, verbose=True))

SVD : Test Set
RMSE: 0.8418
0.8418178627969305
MAE:  0.6531
0.6531010275964316
SVD : Test Set
RMSE: 0.8418
0.8418178627969305
MAE:  0.6531
0.6531010275964316
SVD : Test Set
RMSE: 0.8418
0.8418178627969305
MAE:  0.6531
0.6531010275964316
SVD : Test Set
RMSE: 0.8418
0.8418178627969305
MAE:  0.6531
0.6531010275964316
SVD : Test Set
RMSE: 0.8418
0.8418178627969305
MAE:  0.6531
0.6531010275964316


In [31]:
with open('recommender_predictions.pkl', 'wb') as f:
    pickle.dump(predictions, f, protocol=2)
with open('recommender_algorithm.pkl', 'wb') as f:
    pickle.dump(algo, f, protocol=2)

In [34]:
algo = pd.read_pickle('recommender_algorithm.pkl')

In [37]:
userid = 69976309
# Slice out books they haven't read
fantasy_books = list(book_titles.keys())
user_books = list(df['bookid'][df['userid'] == userid].drop_duplicates())
books_not_read = list(set(fantasy_books) - set(user_books))

In [38]:
estimates = []
for bookid in books_not_read:
    uid = userid
    iid = bookid   
    estimates.append(algo.predict(uid, iid, verbose=False))

In [42]:
book_recs = pd.DataFrame(estimates).sort_values(by='est', ascending = False)
book_recs['title'] = book_recs['iid'].map(book_titles)
book_recs[book_recs.est>4]

,uid,iid,r_ui,est,details,title
25029,69976309,2387969,None,4.948554,{'was_impossible': False},"Nana, Vol. 12 (Nana, #12)"
24753,69976309,17554595,None,4.938754,{'was_impossible': False},"Senlin Ascends (The Books of Babel, #1)"
27476,69976309,19030845,None,4.906374,{'was_impossible': False},Batman: The Dark Knight Returns #1
11852,69976309,962821,None,4.904411,{'was_impossible': False},"The Winter Prince (The Lion Hunters, #1)"
35284,69976309,96259,None,4.894440,{'was_impossible': False},Crush
10215,69976309,62831,None,4.872602,{'was_impossible': False},The Classic Fairy Tales
43047,69976309,11543195,None,4.861885,{'was_impossible': False},"Words of Radiance (The Stormlight Archive, #2)"
35217,69976309,511240,None,4.854214,{'was_impossible': False},"Yotsuba&!, Vol. 1 (Yotsuba&! #1)"
38751,69976309,29748925,None,4.839568,{'was_impossible': False},"Strange the Dreamer (Strange the Dreamer, #1)"
12197,69976309,23435269,None,4.839172,{'was_impossible': False},"The Price of Valor (The Shadow Campaigns, #3)"


,uid,iid,r_ui,est,details,title
25029,69976309,2387969,None,4.948554,{'was_impossible': False},"Nana, Vol. 12 (Nana, #12)"
24753,69976309,17554595,None,4.938754,{'was_impossible': False},"Senlin Ascends (The Books of Babel, #1)"
27476,69976309,19030845,None,4.906374,{'was_impossible': False},Batman: The Dark Knight Returns #1
11852,69976309,962821,None,4.904411,{'was_impossible': False},"The Winter Prince (The Lion Hunters, #1)"
35284,69976309,96259,None,4.894440,{'was_impossible': False},Crush
10215,69976309,62831,None,4.872602,{'was_impossible': False},The Classic Fairy Tales
43047,69976309,11543195,None,4.861885,{'was_impossible': False},"Words of Radiance (The Stormlight Archive, #2)"
35217,69976309,511240,None,4.854214,{'was_impossible': False},"Yotsuba&!, Vol. 1 (Yotsuba&! #1)"
38751,69976309,29748925,None,4.839568,{'was_impossible': False},"Strange the Dreamer (Strange the Dreamer, #1)"
12197,69976309,23435269,None,4.839172,{'was_impossible': False},"The Price of Valor (The Shadow Campaigns, #3)"


,uid,iid,r_ui,est,details,title
25029,69976309,2387969,None,4.948554,{'was_impossible': False},"Nana, Vol. 12 (Nana, #12)"
24753,69976309,17554595,None,4.938754,{'was_impossible': False},"Senlin Ascends (The Books of Babel, #1)"
27476,69976309,19030845,None,4.906374,{'was_impossible': False},Batman: The Dark Knight Returns #1
11852,69976309,962821,None,4.904411,{'was_impossible': False},"The Winter Prince (The Lion Hunters, #1)"
35284,69976309,96259,None,4.894440,{'was_impossible': False},Crush
10215,69976309,62831,None,4.872602,{'was_impossible': False},The Classic Fairy Tales
43047,69976309,11543195,None,4.861885,{'was_impossible': False},"Words of Radiance (The Stormlight Archive, #2)"
35217,69976309,511240,None,4.854214,{'was_impossible': False},"Yotsuba&!, Vol. 1 (Yotsuba&! #1)"
38751,69976309,29748925,None,4.839568,{'was_impossible': False},"Strange the Dreamer (Strange the Dreamer, #1)"
12197,69976309,23435269,None,4.839172,{'was_impossible': False},"The Price of Valor (The Shadow Campaigns, #3)"


,uid,iid,r_ui,est,details,title
25029,69976309,2387969,None,4.948554,{'was_impossible': False},"Nana, Vol. 12 (Nana, #12)"
24753,69976309,17554595,None,4.938754,{'was_impossible': False},"Senlin Ascends (The Books of Babel, #1)"
27476,69976309,19030845,None,4.906374,{'was_impossible': False},Batman: The Dark Knight Returns #1
11852,69976309,962821,None,4.904411,{'was_impossible': False},"The Winter Prince (The Lion Hunters, #1)"
35284,69976309,96259,None,4.894440,{'was_impossible': False},Crush
10215,69976309,62831,None,4.872602,{'was_impossible': False},The Classic Fairy Tales
43047,69976309,11543195,None,4.861885,{'was_impossible': False},"Words of Radiance (The Stormlight Archive, #2)"
35217,69976309,511240,None,4.854214,{'was_impossible': False},"Yotsuba&!, Vol. 1 (Yotsuba&! #1)"
38751,69976309,29748925,None,4.839568,{'was_impossible': False},"Strange the Dreamer (Strange the Dreamer, #1)"
12197,69976309,23435269,None,4.839172,{'was_impossible': False},"The Price of Valor (The Shadow Campaigns, #3)"


,uid,iid,r_ui,est,details,title
25029,69976309,2387969,None,4.948554,{'was_impossible': False},"Nana, Vol. 12 (Nana, #12)"
24753,69976309,17554595,None,4.938754,{'was_impossible': False},"Senlin Ascends (The Books of Babel, #1)"
27476,69976309,19030845,None,4.906374,{'was_impossible': False},Batman: The Dark Knight Returns #1
11852,69976309,962821,None,4.904411,{'was_impossible': False},"The Winter Prince (The Lion Hunters, #1)"
35284,69976309,96259,None,4.894440,{'was_impossible': False},Crush
10215,69976309,62831,None,4.872602,{'was_impossible': False},The Classic Fairy Tales
43047,69976309,11543195,None,4.861885,{'was_impossible': False},"Words of Radiance (The Stormlight Archive, #2)"
35217,69976309,511240,None,4.854214,{'was_impossible': False},"Yotsuba&!, Vol. 1 (Yotsuba&! #1)"
38751,69976309,29748925,None,4.839568,{'was_impossible': False},"Strange the Dreamer (Strange the Dreamer, #1)"
12197,69976309,23435269,None,4.839172,{'was_impossible': False},"The Price of Valor (The Shadow Campaigns, #3)"


**End Notebook**

***